In [2]:
!pip install -qq -U spacy
!pip install -qq spacy-transformers

In [3]:
%%writefile base_config.cfg
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null

[system]
gpu_allocator = "pytorch"

[nlp]
lang = "bn"
pipeline = ["transformer","ner"]
batch_size = 128

[components]

[components.transformer]
factory = "transformer"

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v1"
name = "bert-base-multilingual-uncased"
tokenizer_config = {"use_fast": true}

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.ner.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 5e-5

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256

[initialize]
vectors = ${paths.vectors}

Writing base_config.cfg


In [4]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

2021-08-28 17:19:08.064534: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
!python -m spacy train ./config.cfg \
    --gpu-id 0 \
    --output models_multilingual_bert \
    --paths.train ../input/ner-data-9k/train_all.spacy \
    --paths.dev ../input/ner-data-9k/val_all.spacy

2021-08-28 17:19:32.626472: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Created output directory: models_sahaj_bert
ℹ Saving to output directory: models_sahaj_bert
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-28 17:19:36,479] [INFO] Set up nlp object from config
[2021-08-28 17:19:36,491] [INFO] Pipeline: ['transformer', 'ner']
[2021-08-28 17:19:36,496] [INFO] Created vocabulary
[2021-08-28 17:19:36,498] [INFO] Finished initializing nlp object
Downloading: 100%|██████████████████████████████| 625/625 [00:00<00:00, 637kB/s]
Downloading: 100%|███████████████████████████| 872k/872k [00:00<00:00, 8.26MB/s]
Downloading: 100%|█████████████████████████| 1.72M/1.72M [00:00<00:00, 12.3MB/s]
Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 24.5kB/s]
Downloading: 100%|███████████████████████████| 672M/672M [00:14<00:00, 46.2MB/s]
Some weights o

In [12]:
!zip -r ./models_multilungiul_bert.zip ./models_multilingual_bert/model-best

  adding: models_sahaj_bert/model-best/ (stored 0%)
  adding: models_sahaj_bert/model-best/tokenizer (deflated 82%)
  adding: models_sahaj_bert/model-best/config.cfg (deflated 60%)
  adding: models_sahaj_bert/model-best/meta.json (deflated 57%)
  adding: models_sahaj_bert/model-best/ner/ (stored 0%)
  adding: models_sahaj_bert/model-best/ner/cfg (deflated 33%)
  adding: models_sahaj_bert/model-best/ner/model (deflated 8%)
  adding: models_sahaj_bert/model-best/ner/moves (deflated 39%)
  adding: models_sahaj_bert/model-best/vocab/ (stored 0%)
  adding: models_sahaj_bert/model-best/vocab/strings.json (deflated 90%)
  adding: models_sahaj_bert/model-best/vocab/vectors (deflated 45%)
  adding: models_sahaj_bert/model-best/vocab/key2row (stored 0%)
  adding: models_sahaj_bert/model-best/vocab/lookups.bin (stored 0%)
  adding: models_sahaj_bert/model-best/transformer/ (stored 0%)
  adding: models_sahaj_bert/model-best/transformer/cfg (stored 0%)
  adding: models_sahaj_bert/model-best/transfo

In [18]:
import spacy

nlp = spacy.load("./models_multilingual_bert/model-best")

text_list = [
    "আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম",
    "১০০ টাকা জমা দিয়েছেন কবির",
    "ডিপিডিসির স্পেশাল টাস্কফোর্সের প্রধান মুনীর চৌধুরী জানান",
    "অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পরীক্ষার প্রশ্নপত্র ফাঁসের অভিযোগ উঠেছে।",
    "সে আজকে ঢাকা যাবে",
]
for text in text_list:
    doc = nlp(text)

    print(f"Input: {text}")
    for entity in doc.ents:
        print(f"Entity: {entity.text}, Label: {entity.label_}")
    print("---")


Input: আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম
Entity: আব্দুর রহিম, Label: PER
---
Input: ১০০ টাকা জমা দিয়েছেন কবির
Entity: কবির, Label: PER
---
Input: ডিপিডিসির স্পেশাল টাস্কফোর্সের প্রধান মুনীর চৌধুরী জানান
Entity: মুনীর চৌধুরী, Label: PER
---
Input: অগ্রণী ব্যাংকের জ্যেষ্ঠ কর্মকর্তা পদে নিয়োগ পরীক্ষার প্রশ্নপত্র ফাঁসের অভিযোগ উঠেছে।
---
Input: সে আজকে ঢাকা যাবে
---
